In [7]:
import pandas as pd
import numpy as np


# === Read the data
df_train = pd.read_csv("processed_train.csv",  dtype={"id": str})
df_test = pd.read_csv("processed_test.csv",  dtype={"id": str})

# === Extract photo
df_train["pixels"] = df_train["pixels"].apply(eval)
df_test["pixels"] = df_test["pixels"].apply(eval)


In [10]:
# === Subtask 1
subtask1_rows = []
vectors = []
for i, row in df_train.iterrows():
    vector = np.array(row["pixels"])
    vectors.append(vector)

global_mean_vector = np.mean(vectors, axis=0)

for i, row in df_train.iterrows():
    centered_output = np.array(row["pixels"]) - global_mean_vector
    subtask1_rows.append((1, row["id"], centered_output))

(1, '015002', array([-0.10046729, -0.10046729, -0.10046729, ..., -0.57476636,
       -0.66238318, -0.43457944]))
(1, '015003', array([-0.10046729, -0.10046729, -0.10046729, ..., -0.57476636,
       -0.66238318, -0.43457944]))


In [28]:
# === Subtask 2
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train = np.vstack(df_train["pixels"].values)
y_train = df_train["class"]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

clf = ExtraTreesClassifier(random_state=42)
clf.fit(X_train, y_train)

y_val_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 0.8662790697674418


- HistGradientBoostingClassifier - 0.8546511627906976
- AdaBoost - 0.8604651162790697
- ExtraTrees - 0.8662790697674418

In [29]:
X_test = np.vstack(df_test["pixels"].values)
y_pred = clf.predict(X_test)

subtask2_rows = []
for id_, pred in zip(df_test["id"], y_pred):
    subtask2_rows.append((2, id_, pred))


In [30]:
# === Final submission
submission_rows = subtask1_rows + subtask2_rows
df_submission = pd.DataFrame(submission_rows, columns=["subtaskID", "datapointID", "answer"])
df_submission.to_csv("submission.csv", index=False)